In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
stations = pd.read_csv("../Data/allstations.csv")

/Users/christianopperman/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3,6,7,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
stations.columns

Index(['dock_id', 'dock_name', 'date', 'hour', 'minute', 'pm', 'avail_bikes',
       'avail_docks', 'tot_docks', '_lat', '_long', 'in_service',
       'status_key'],
      dtype='object')

In [3]:
stations_processed = stations.copy()

##### Drop rows where the values don't make sense or are incompatible

In [4]:
print(f"Dropping rows, current table shape: {stations_processed.shape}")
stations_processed.dropna(inplace = True)
stations_processed.drop(stations_processed[stations_processed['dock_id'].apply(
    lambda x: isinstance(x, str))].index, inplace = True)
stations_processed = stations_processed[stations_processed['tot_docks']<2000]
print(f"Finished dropping rows, new table shape: {stations_processed.shape}")

Dropping rows, current table shape: (35340507, 13)
Finished dropping rows, new table shape: (35143894, 13)


##### Parsing available bikes and docks

In [5]:
# Parsing availabile bikes
mask = ~stations_processed['avail_bikes'].astype(str).str.contains('[A-z]')
stations_processed = stations_processed[mask]
stations_processed['avail_bikes'] = stations_processed['avail_bikes'].apply(lambda x: re.sub("\"", "", str(x))) # Remove quotation marks so strings can be converted to integers later
stations_processed = stations_processed[stations_processed['avail_bikes']!=""] # Drop any empty values
stations_processed['avail_bikes'] = stations_processed['avail_bikes'].astype(float).astype(int) # Convert strings to integers
stations_processed = stations_processed[stations_processed['avail_bikes']<=100] # Remove any row with an impossible number of bikes

# Parsing available docks
mask = ~stations_processed['avail_docks'].astype(str).str.contains('[A-z]')
stations_processed = stations_processed[mask]
stations_processed['avail_docks'] = stations_processed['avail_docks'].apply(lambda x: re.sub("\"", "", str(x))) # Remove quotation marks so strings can be converted to integers later
stations_processed = stations_processed[stations_processed['avail_docks']!=""] # Drop any empty values
stations_processed['avail_docks'] = stations_processed['avail_docks'].astype(float).astype(int) # Convert strings to integers
stations_processed = stations_processed[stations_processed['avail_docks']<=100] # Remove any row with an impossible number of bikes

In [6]:
stations_processed.shape

(35143887, 13)

##### Parse date column into datetime format

In [7]:
stations_processed['date'] = pd.to_datetime(stations_processed['date'], format='"%y-%m-%d"')

##### Convert numeric columns from strings to integers/floats as appropriate

In [8]:
stations_processed['dock_id'] = stations_processed['dock_id'].astype(int)
stations_processed['tot_docks'] = stations_processed['tot_docks'].astype(int)
stations_processed['minute'] = stations_processed['minute'].astype(int)

stations_processed['_lat'] = stations_processed['_lat'].apply(lambda x: float(re.sub('\"', "", str(x))))
stations_processed['_long'] = stations_processed['_long'].apply(
    lambda x: re.sub('[^-^.0-9]', "", str(x))).apply(lambda x: re.sub("-{2}", "-", str(x)))
stations_processed = stations_processed[stations_processed['_long']!=""]
stations_processed['_long'].astype(float)

stations_processed['hour'] = stations_processed['hour'].apply(lambda x: re.sub('[^0-9]', "", str(x))).astype(int)
stations_processed['hour'].loc[stations_processed['pm']==1] = stations_processed['hour'].loc[stations_processed['pm']==1] + 12 # Convert hours to 24-hour time

/Users/christianopperman/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


##### Convert minutes to half hour increments

In [29]:
stations_processed['minute'] = stations_processed['minute'].apply(lambda x: '00' if x < 30 else '30')

##### Remove quotations from dock name

In [10]:
stations_processed['dock_name'] = stations_processed['dock_name'].apply(lambda x: str(re.sub('\"', "", x)))

##### Create a depletion status column

In [11]:
stations_processed['depletion_status'] = (stations_processed['avail_bikes']/stations_processed['tot_docks']).apply(
        lambda x: "Full Risk" if x > 2/3 else "Empty Risk" if x < 1/3 else "Healthy")

##### Drop unnecessary columns

In [12]:
stations_processed.drop(['pm', 'in_service', 'status_key'], axis=1, inplace = True)

In [13]:
stations_processed.head()

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,depletion_status
0,116,W 17 St & 8 Ave,2015-04-02,14,30,5,32,32,40.741776,-74.00149746,Empty Risk
1,116,W 17 St & 8 Ave,2015-04-02,15,0,1,36,36,40.741776,-74.00149746,Empty Risk
2,116,W 17 St & 8 Ave,2015-04-02,15,0,2,35,35,40.741776,-74.00149746,Empty Risk
3,116,W 17 St & 8 Ave,2015-04-02,15,0,3,34,34,40.741776,-74.00149746,Empty Risk
4,116,W 17 St & 8 Ave,2015-04-02,15,30,2,35,35,40.741776,-74.00149746,Empty Risk


In [14]:
stations_processed.shape

(35143886, 11)

In [15]:
stations_processed.dtypes

dock_id                      int64
dock_name                   object
date                datetime64[ns]
hour                         int64
minute                       int64
avail_bikes                  int64
avail_docks                  int64
tot_docks                    int64
_lat                       float64
_long                       object
depletion_status            object
dtype: object

In [ ]:
df['season']=df['starttime'].dt.month.apply(lambda x: 'winter' if x <=2 else 'spring' if x<=5 else 
                                              'summer' if x<=8 else 'fall' if x<=11 else 'winter')

In [30]:
stations_processed.head()

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,depletion_status
0,116,W 17 St & 8 Ave,2015-04-02,14,30,5,32,32,40.741776,-74.00149746,Empty Risk
1,116,W 17 St & 8 Ave,2015-04-02,15,00,1,36,36,40.741776,-74.00149746,Empty Risk
2,116,W 17 St & 8 Ave,2015-04-02,15,00,2,35,35,40.741776,-74.00149746,Empty Risk
3,116,W 17 St & 8 Ave,2015-04-02,15,00,3,34,34,40.741776,-74.00149746,Empty Risk
4,116,W 17 St & 8 Ave,2015-04-02,15,30,2,35,35,40.741776,-74.00149746,Empty Risk


In [39]:
stations_processed = stations_processed.assign(time = lambda x: x['hour'].astype(str) + ":" + x['minute'].astype(str))
stations_processed = stations_processed.assign(dayofweek = lambda x: x['date'].dt.weekday)
stations_processed = stations_processed.assign(season = lambda x: x['date'].dt.month.apply(
    lambda y: 'winter' if y <= 2 else 'spring' if y <= 5 else 'summer' if y <= 8 else 'fall' if y <= 11 else 'winter'))

In [42]:
stations_processed.head()

,dock_id,dock_name,date,hour,minute,avail_bikes,avail_docks,tot_docks,_lat,_long,depletion_status,time,dayofweek,season
0,116,W 17 St & 8 Ave,2015-04-02,14,30,5,32,32,40.741776,-74.00149746,Empty Risk,14:30,3,spring
1,116,W 17 St & 8 Ave,2015-04-02,15,00,1,36,36,40.741776,-74.00149746,Empty Risk,15:00,3,spring
2,116,W 17 St & 8 Ave,2015-04-02,15,00,2,35,35,40.741776,-74.00149746,Empty Risk,15:00,3,spring
3,116,W 17 St & 8 Ave,2015-04-02,15,00,3,34,34,40.741776,-74.00149746,Empty Risk,15:00,3,spring
4,116,W 17 St & 8 Ave,2015-04-02,15,30,2,35,35,40.741776,-74.00149746,Empty Risk,15:30,3,spring


In [41]:
stations_processed.to_csv("/Users/christianopperman/Dropbox/CitiBike Capstone/allstations_processed.csv", index = False)